In [6]:
import sys
sys.path.insert(0, r"D:\ETUDEµ\CITE\ETAPE4\PROJET DE FIN DE SESSION\TOURMANT 1")

from db.connection import init_db
from db.repositories.detection_repo import DetectionRepository
from feature_extraction2.sliding_window import SlidingWindow
from datetime import datetime


#  INITIALISATION DB (OBLIGATOIRE)
init_db()

detection_repo = DetectionRepository()

sliding_window = SlidingWindow(
    detection_repository=detection_repo,
    window_duration=5.0,
    step=1.0
)
for window_df in sliding_window.generate(
    camera_ids=[1],
    start_time=datetime(2026, 1, 16, 18, 36, 35),
    end_time=datetime(2026, 1, 16, 18, 38, 35)
):
    print(window_df)
   

D:\ETUDEµ\CITE\ETAPE4\PROJET DE FIN DE SESSION\TOURMANT 1\db\repositories\detection_repo.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)


In [4]:
window_df

,id,camera_id,timestamp,object_class,track_id,bbox_x,bbox_y,bbox_w,bbox_h,confidence


Ce code permet de tester toutes les extractions de caractéristique 
Très important 

In [1]:
# ==========================================================
# IMPORTS & PATH
# ==========================================================
import sys
sys.path.insert(0, r"D:\ETUDEµ\CITE\ETAPE4\PROJET DE FIN DE SESSION\TOURMANT 1")

from datetime import datetime

from db.connection import init_db
from db.repositories.detection_repo import DetectionRepository

from feature_extraction2.sliding_window import SlidingWindow
from feature_extraction2.human_features import HumanPresenceFeatures
from feature_extraction2.epi_features import EPIFeatures
from feature_extraction2.machine_features import MachineVehicleFeatures
from feature_extraction2.proximity_features import ProximityFeatures
from feature_extraction2.temporal_dynamics_features import TemporalDynamicsFeatures

from db.repositories.zone_repo import ZoneRepository
from db.repositories.camera_calibration_repo import CameraCalibrationRepository
from feature_extraction2.zone_features import ZoneFeatures
from feature_extraction2.activity_stability_features import ActivityStabilityFeatures
from feature_extraction2.activity_transition_features import ActivityTransitionFeatures
from feature_extraction2.activity_stability_features import ActivityStabilityFeatures






# ==========================================================
# INITIALISATION DB (OBLIGATOIRE) ET REQUETTE
# ==========================================================
init_db()

detection_repo = DetectionRepository()


zone_repo = ZoneRepository()
calib_repo = CameraCalibrationRepository()

zones = zone_repo.get_active_zones_by_plan(plan_id=1)
H = calib_repo.get_active_calibration(camera_id=1, plan_id=1)


# ==========================================================
# SLIDING WINDOW
# ==========================================================
sliding_window = SlidingWindow(
    detection_repository=detection_repo,
    window_duration=5.0,   # fenêtre de 5 secondes
    step=1.0               # glissement de 1 seconde
)


# ==========================================================
# FEATURE EXTRACTORS ( NE PAS METTRE DANS LA BOUCLE)
# ==========================================================
human_extractor = HumanPresenceFeatures(
    image_width=1920,
    image_height=1080,
    stationary_threshold=2.0
)

epi_extractor = EPIFeatures(
    window_duration=5.0,
    persistent_threshold=3
)

machine_extractor = MachineVehicleFeatures(
    window_duration=5.0
)

proximity_extractor = ProximityFeatures(
    window_duration=5.0,
    danger_distance_pp=100,
    danger_distance_pm=150
)

temporal_extractor = TemporalDynamicsFeatures(window_duration=5.0)


zone_extractor = ZoneFeatures(
    zones=zones,
    homography=H,
    window_duration=5.0
)

stability_extractor = ActivityStabilityFeatures(history_size=5)


# ==========================================================
# PARAMÈTRES DE TEST
# ==========================================================
START_TIME = datetime(2026, 1, 18, 20, 31, 40)
END_TIME   = datetime(2026, 1, 18, 20, 32, 25)

MAX_WINDOWS = 10   # pour ne pas inonder le notebook


# ==========================================================
# TEST PIPELINE COMPLET
# ==========================================================
for i, window_df in enumerate(
    sliding_window.generate(
        camera_ids=[1],
        start_time=START_TIME,
        end_time=END_TIME
    )
):
    if i >= MAX_WINDOWS:
        break

    print("\n" + "=" * 60)
    print(f"🪟 FENÊTRE {i}")
    print("=" * 60)

    print(f"Nombre total de détections : {len(window_df)}")

    if window_df.empty:
        print(" Fenêtre vide")
        continue

    # ------------------------------------------------------
    # DEBUG TEMPOREL
    # ------------------------------------------------------
    print(
        " Intervalle temporel :",
        window_df["timestamp"].min(),
        "→",
        window_df["timestamp"].max()
    )

    # ------------------------------------------------------
    # OBJETS DÉTECTÉS
    # ------------------------------------------------------
    print("\n Objets détectés :")
    print(window_df["object_class"].value_counts())

    # ======================================================
    # EPI FEATURES
    # ======================================================
    print("\n EPI FEATURES")
    epi_features = epi_extractor.extract(window_df)

    for k, v in epi_features.items():
        print(f"{k:<35} : {v}")

    # ======================================================
    # MACHINE / VEHICLE FEATURES
    # ======================================================
    print("\n MACHINE / VEHICLE FEATURES")
    machine_features = machine_extractor.extract(window_df)

    for k, v in machine_features.items():
        print(f"{k:<35} : {v}")


    # ======================================================
    # Proximité
    # ======================================================
    print("\n Proximité PERSONNE / VEHICLE FEATURES")
    proximity_features  = proximity_extractor.extract(window_df)

    for k, v in proximity_features .items():
        print(f"{k:<35} : {v}")



    # ======================================================
    # temporel
    # ======================================================
    print("\n Temporel")
    temporal_features = temporal_extractor.extract(window_df)
    for k,v in temporal_features.items():
        print(f"{k}: {v}")



    # ======================================================
    # Zone 
    # ======================================================
    print("\n Zone et homographie")
    zone_features = zone_extractor.extract(window_df)
    for k,v in zone_features.items():
        print(f"{k}: {v}")




    

D:\ETUDEµ\CITE\ETAPE4\PROJET DE FIN DE SESSION\TOURMANT 1\db\repositories\detection_repo.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)



🪟 FENÊTRE 0
Nombre total de détections : 111
 Intervalle temporel : 2026-01-18 20:31:40 → 2026-01-18 20:31:44

 Objets détectés :
object_class
safety_vest    59
person         45
hardhat         6
safety_cone     1
Name: count, dtype: int64

 EPI FEATURES
helmet_compliance_ratio             : 1.0
vest_compliance_ratio               : 1.0
mask_compliance_ratio               : 0.0
avg_epi_compliance                  : 0.6666666666666666
num_no_helmet_events                : 0
num_no_vest_events                  : 0
num_no_mask_events                  : 0
epi_violation_duration              : 0.0
persistent_epi_violation            : 0

 MACHINE / VEHICLE FEATURES
num_machines_avg                    : 0.0
machine_presence_ratio              : 0.0
unique_machine_tracks               : 0
machine_type_ratio                  : 0.0
co_presence_person_machine          : 0.0
machine_dominance_ratio             : 0.0
machine_only_duration               : 0.0

 Proximité PERSONNE / VEHICLE FEATUR

In [2]:
# ==========================================================
# PARAMÈTRES DE TEST
# ==========================================================
START_TIME = datetime(2026, 1, 18, 20, 31, 40)
END_TIME   = datetime(2026, 1, 18, 20, 32, 25)

MAX_WINDOWS = 10   # pour ne pas inonder le notebook


from feature_extraction2.pipeline import extract_features_pipeline
import pandas as pd

df_all, df_ml = extract_features_pipeline(
    sliding_window=sliding_window,
    human_extractor=human_extractor,
    epi_extractor=epi_extractor,
    machine_extractor=machine_extractor,
    proximity_extractor=proximity_extractor,
    temporal_extractor=temporal_extractor,
    zone_extractor=zone_extractor,
    stability_extractor=stability_extractor,
    camera_ids=[1],
    start_time=START_TIME,
    end_time=END_TIME,
    max_windows=10
)

df_all=pd.DataFrame(df_all)
df_ml=pd.DataFrame(df_ml)

print("\n ALL FEATURES")
df_all.head()


D:\ETUDEµ\CITE\ETAPE4\PROJET DE FIN DE SESSION\TOURMANT 1\db\repositories\detection_repo.py:42: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, params=params)



 ALL FEATURES


,window_index,window_start,window_end,camera_id,num_detections,avg_num_persons,max_num_persons,unique_person_tracks,person_density,person_presence_ratio,...,time_in_zone,multiple_zone_exposure,num_people_in_high_risk_zone,proportion_time_in_high_risk_zone,zone_entry_exit_events,first_entry_time,last_exit_time,activity_persistence_score,motion_consistency,motion_stability_score
0,0,2026-01-18 20:31:40,2026-01-18 20:31:44,1,111,2.0,2,2,9.645062e-07,1.0,...,0,0,0,0.0,0,None,None,0.500000,0.500000,0.500000
1,1,2026-01-18 20:31:41,2026-01-18 20:31:45,1,111,2.0,2,2,9.645062e-07,1.0,...,0,0,0,0.0,0,None,None,1.000000,0.994797,0.000298
2,2,2026-01-18 20:31:42,2026-01-18 20:31:46,1,101,1.8,2,2,8.680556e-07,1.0,...,0,0,0,0.0,0,None,None,0.913842,0.909061,0.000059
3,3,2026-01-18 20:31:43,2026-01-18 20:31:47,1,92,1.6,2,2,7.716049e-07,1.0,...,0,0,0,0.0,0,None,None,0.857757,0.917573,0.000016
4,4,2026-01-18 20:31:44,2026-01-18 20:31:48,1,67,1.5,2,2,7.233796e-07,0.8,...,0,0,0,0.0,0,None,None,0.830592,0.835213,0.000018


In [3]:
df_ml.head()

,avg_num_persons,max_num_persons,unique_person_tracks,person_density,person_presence_ratio,avg_epi_compliance,helmet_compliance_ratio,vest_compliance_ratio,num_no_helmet_events,num_no_vest_events,...,direction_variance,num_persons_in_zone,num_people_in_high_risk_zone,proportion_time_in_high_risk_zone,multiple_zone_exposure,avg_person_height,time_near_machine,num_machine_interactions,motion_consistency,activity_persistence_score
0,2.0,2,2,9.645062e-07,1.0,0.666667,1.0,1.0,0,0,...,4719.832666,0,0,0.0,0,52.727584,0.0,0,0.500000,0.500000
1,2.0,2,2,9.645062e-07,1.0,0.333333,0.0,1.0,0,0,...,4527.510389,0,0,0.0,0,54.454152,0.0,0,0.994797,1.000000
2,1.8,2,2,8.680556e-07,1.0,0.666667,1.0,1.0,0,0,...,7594.897774,0,0,0.0,0,56.490874,0.0,0,0.909061,0.913842
3,1.6,2,2,7.716049e-07,1.0,0.666667,1.0,1.0,0,0,...,14542.594041,0,0,0.0,0,59.811300,0.0,0,0.917573,0.857757
4,1.5,2,2,7.233796e-07,0.8,0.666667,1.0,1.0,0,0,...,11207.106432,0,0,0.0,0,62.883533,0.0,0,0.835213,0.830592


In [4]:
df_all.columns

Index(['window_index', 'window_start', 'window_end', 'camera_id',
       'num_detections', 'avg_num_persons', 'max_num_persons',
       'unique_person_tracks', 'person_density', 'person_presence_ratio',
       'num_person_entries', 'avg_person_height', 'stationary_person_ratio',
       'crowding_score', 'helmet_compliance_ratio', 'vest_compliance_ratio',
       'mask_compliance_ratio', 'avg_epi_compliance', 'num_no_helmet_events',
       'num_no_vest_events', 'num_no_mask_events', 'epi_violation_duration',
       'persistent_epi_violation', 'num_machines_avg',
       'machine_presence_ratio', 'unique_machine_tracks', 'machine_type_ratio',
       'co_presence_person_machine', 'machine_dominance_ratio',
       'machine_only_duration', 'avg_person_person_distance',
       'std_person_person_distance', 'avg_person_machine_distance',
       'min_person_person_distance', 'min_person_machine_distance',
       'dangerous_proximity_frames', 'bbox_overlap_person_machine',
       'time_near_machi

In [5]:
df_ml.columns

Index(['avg_num_persons', 'max_num_persons', 'unique_person_tracks',
       'person_density', 'person_presence_ratio', 'avg_epi_compliance',
       'helmet_compliance_ratio', 'vest_compliance_ratio',
       'num_no_helmet_events', 'num_no_vest_events', 'num_machines_avg',
       'machine_presence_ratio', 'unique_machine_tracks',
       'co_presence_person_machine', 'dangerous_proximity_frames',
       'min_person_machine_distance', 'avg_person_speed', 'std_person_speed',
       'sudden_stop_events', 'erratic_motion_score', 'direction_variance',
       'num_persons_in_zone', 'num_people_in_high_risk_zone',
       'proportion_time_in_high_risk_zone', 'multiple_zone_exposure',
       'avg_person_height', 'time_near_machine', 'num_machine_interactions',
       'motion_consistency', 'activity_persistence_score'],
      dtype='object')

In [7]:
df_ml.shape

(10, 25)

In [3]:
import cv2
img = cv2.imread("utils/8-768x577.jpg")
cv2.imshow("img", img)
cv2.waitKey(0)

-1

**Option 1 — Homographie**

Principe

Tu relies l’image caméra au plan par une matrice 3×3

Cette matrice est calculée une fois par caméra

Données nécessaires (calibration)

Pour chaque caméra :

4 points minimum visibles :

(x_image, y_image) ↔ (x_plan, y_plan)

In [4]:
import numpy as np
import cv2

def compute_homography(pts_image, pts_plan):
    """
    pts_image : liste ou np.array Nx2 (pixels caméra)
    pts_plan  : liste ou np.array Nx2 (coordonnées plan)
    """

    # Conversion en numpy float32 (OBLIGATOIRE pour OpenCV)
    pts_image = np.array(pts_image, dtype=np.float32)
    pts_plan = np.array(pts_plan, dtype=np.float32)

    assert pts_image.shape == pts_plan.shape
    assert pts_image.shape[0] >= 4

    # Calcul de l'homographie
    H, mask = cv2.findHomography(
        pts_image,
        pts_plan,
        method=cv2.RANSAC,      # robuste aux erreurs
        ransacReprojThreshold=3.0
    )

    return H, mask


In [5]:
pts_image = [
    [305, 331],
    [193, 313],
    [331, 328],
    [476, 355]
]

pts_plan = [
    [12.0, 6.0],
    [10.0, 5.5],
    [13.2, 6.1],
    [15.0, 7.0]
]

H, inliers = compute_homography(pts_image, pts_plan)

print("Homographie :\n", H)
print("Points valides (RANSAC) :", inliers.ravel())


Homographie :
 [[ 6.73916337e-03 -4.53003790e-02  1.27985312e+01]
 [ 3.23974141e-03 -2.16205477e-02  6.09805545e+00]
 [ 5.37614690e-04 -3.55189304e-03  1.00000000e+00]]
Points valides (RANSAC) : [1 1 1 1]


Pour interférence. 

In [ ]:
import torch 
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size,num_layers):
        super().__init__()
        self.lstm=nn.LSTM(input_size, hidden_size,num_layers=num_layers,batch_first=True)
        self.fc=nn.Linear(hidden_size,)